In [62]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [63]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [64]:
# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open('smartwaiver_test').sheet1

In [65]:
# put the full file name for the smartwaiver data here
smartwaiver = 'smartwaiver-172990-647267ce02afe.csv'

# load the data from your CSV file
data = pd.read_csv(smartwaiver)

In [66]:
# convert 'Waiver Date' column to datetime
data['Date Completed (UTC)'] = pd.to_datetime(data['Date Completed (UTC)'])

In [67]:
# get the current date
now = datetime.now()

# calculate the date one year ago
one_year_ago = now - timedelta(days=365)

In [68]:
# filter the data to get volunteers who signed a waiver in the past year
filtered_data = data[data['Date Completed (UTC)'] >= one_year_ago]

In [69]:
filtered_data.head(5)

,First,Middle,Last,Date of Birth,Date Completed (UTC),Waiver ID,Status,Check-ins,Gender,Phone,...,Parent/Guardian First Name,Parent/Guardian Middle Name,Parent/Guardian Last Name,Parent/Guardian Phone,Parent/Guardian Relationship,Email,signing_type,template_type,Title of Document,typed_font
0,Rebekah,NaN,Vickrey,NaN,2023-04-21 00:48:37,vhDkW4Rq5SJG6uNA8NSCVM,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,corey.vickrey@gmail.com,draw,1,SCMTS + City of Santa Cruz,dancing
1,Mark,NaN,Esguerra,NaN,2023-04-21 00:52:43,TAWhPFPFT5qXywKe2CtRKK,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,m.esguerra@gmail.com,draw,1,SCMTS + City of Santa Cruz,dancing
2,Jeff,NaN,Wurz,NaN,2023-04-23 03:19:41,N4g2auUdKCqoWJd327mxUC,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,jeff.wurz@gmail.com,draw,1,SCMTS + City of Santa Cruz,dancing
3,Shay,NaN,Mossing,NaN,2023-04-24 22:05:22,EM3PmX9xBWXmhLPvpA6dBN,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,scmossing@yahoo.com,draw,1,SCMTS + City of Santa Cruz,dancing
4,Vance,NaN,Turner,NaN,2023-04-26 13:23:54,Y9juxBErL6jWGxwuJYShvG,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,vanceturner123@gmail.com,draw,1,SCMTS + City of Santa Cruz,dancing


In [70]:
# Get all records from the Google Sheet
records = sheet.get_all_records()

# Create an empty DataFrame to store unmatched names
unmatched_names = pd.DataFrame(columns=['First Name', 'Last Name', 'Email'])

In [71]:

# iterate over the filtered data
for index, row in filtered_data.iterrows():
    
    # find the volunteer in the Google Spreadsheet
    matched_records = [record for record in records if record['Last'] == row['Last'] and record['First'] == row['First']]

    if matched_records:
          
        # if there is a match check the box
        record = matched_records[0]
        
        row_number = records.index(record) + 2 # Adds 2 because gspread row index starts at 1 and we have a header row

        # check the box in the column next to the volunteer (change 'B' to your actual column)
        sheet.update_cell(row_number, 3, True) # The number 3 reflects the column number

else:
    # if there's no match, add the name to the DataFrame
        unmatched_names = unmatched_names.append({'First': row['First'], 'Last': row['Last']}, ignore_index=True)

In [72]:
unmatched_names

,First Name,Last Name,Email,First,Last
0,NaN,NaN,NaN,Jack,Mudd
